In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [2]:
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Narsaq,GL,2021-04-16 19:50:16,60.9167,-46.0500,23.00,33,20,32.21
1,1,Leningradskiy,RU,2021-04-16 19:50:16,69.3833,178.4167,28.78,81,100,6.46
2,2,Miles City,US,2021-04-16 19:46:00,46.4083,-105.8406,48.20,46,20,11.50
3,3,Hobart,AU,2021-04-16 19:46:14,-42.8794,147.3294,54.00,58,100,8.05
4,4,Punta Arenas,CL,2021-04-16 19:50:17,-53.1500,-70.9167,50.00,53,20,29.93


In [3]:
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [4]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [5]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [6]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]

# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.6)
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=temps, dissipating=False, max_intensity=300, point_radius=4)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
# Heatmap of humidity
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the humidity %.
humidity = city_data_df["Humidity"]
# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.6)
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
# Heatmap of % cloudiness
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the humidity %.
clouds = city_data_df["Cloudiness"]
# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.6)
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=clouds, dissipating=False, max_intensity=300, point_radius=4)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [9]:
# Heatmap of Wind Speed
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the humidity %.
wind_speed = city_data_df["Wind Speed"]
# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.6)
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=wind_speed, dissipating=False, max_intensity=300, point_radius=4)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [10]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 85


In [11]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & 
                                       (city_data_df["Max Temp"] >= min_temp)]


In [12]:
preferred_cities_df.count()

City_ID       158
City          158
Country       157
Date          158
Lat           158
Lng           158
Max Temp      158
Humidity      158
Cloudiness    158
Wind Speed    158
dtype: int64

In [13]:
preferred_cities_df = preferred_cities_df.dropna(subset=['Country'])

In [14]:
preferred_cities_df.count()

City_ID       157
City          157
Country       157
Date          157
Lat           157
Lng           157
Max Temp      157
Humidity      157
Cloudiness    157
Wind Speed    157
dtype: int64

In [15]:
# Create a DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
5,Avarua,CK,82.40,-21.2078,-159.7750,
8,Anloga,GH,84.20,5.7947,0.8973,
12,Rikitea,PF,77.11,-23.1203,-134.9692,
19,Georgetown,MY,80.60,5.4112,100.3354,
23,Gamba,GA,78.91,-2.6500,10.0000,
25,Chimore,BO,80.37,-16.9833,-65.1333,
32,Bambous Virieux,MU,80.01,-20.3428,57.7575,
42,Santa Rosalia,MX,76.66,27.3167,-112.2833,
44,Atuona,PF,80.42,-9.8000,-139.0333,
48,Tukrah,LY,82.69,32.5341,20.5791,


In [17]:
# Set parameters to search for a hotel.
params = {"radius" : 5000, "type" : "lodging", "key" : g_key}

# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [20]:
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
5,Avarua,CK,82.40,-21.2078,-159.7750,Paradise Inn
8,Anloga,GH,84.20,5.7947,0.8973,Pin Drop Hotel
12,Rikitea,PF,77.11,-23.1203,-134.9692,Pension Maro'i
19,Georgetown,MY,80.60,5.4112,100.3354,Cititel Penang
23,Gamba,GA,78.91,-2.6500,10.0000,
25,Chimore,BO,80.37,-16.9833,-65.1333,Bichito
32,Bambous Virieux,MU,80.01,-20.3428,57.7575,Casa Tia Villa
42,Santa Rosalia,MX,76.66,27.3167,-112.2833,Hotel Las Casitas Santa Rosalia
44,Atuona,PF,80.42,-9.8000,-139.0333,Villa Enata
48,Tukrah,LY,82.69,32.5341,20.5791,Lokta restaurant


In [28]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

In [29]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))